In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from sbi.inference import NPE
import torch

2025-03-27 10:36:23.948143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743068183.960929    8290 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743068183.964674    8290 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743068183.974321    8290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743068183.974339    8290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743068183.974341    8290 computation_placer.cc:177] computation placer alr

In [ ]:
ts, dt = (30.0, 0.1)
def amv_simulator(theta):
    m, Iz, xg, Xud, Yvd, Yrd, Nvd, Nrd, Xu, Xuu, Yv, Yvv, Yr, Nv, Nr, Nrr = theta
    m11 = m - Xud
    m22 = m - Yvd
    m23 = m * xg - Yrd
    m32 = m * xg - Nvd
    m33 = Iz - Nrd
    mt = m22*m33-m23*m32

    M = np.array([[m11, 0, 0], [0, m22, m23], [0, m32, m33]])
    B = dt * np.linalg.inv(M)
    
    a1 = dt * (m - Yvd) / (m - Xud)
    a2 = dt * (m * xg - Yrd) / (m - Xud)
    a3 = (-dt * (Iz - Nrd) * (m - Xud) / mt) + (dt * (m * xg - Yrd) * (m * xg - Yrd) / mt)
    a4 = dt * (m * xg - Yrd) * (Xud - Yvd) / mt
    a5 = (dt * (m * xg - Nvd) * (m - Xud) / mt) - (dt * (m - Yvd) * (m * xg - Yrd) / mt)
    a6 = -dt * (m - Yvd) * (Xud - Yvd) / mt

    b1  = dt * Xu / (m - Xud)
    b2  = dt * Xuu / (m - Xud)
    b3  = (dt * (Iz - Nrd) * Yv / mt) - (dt * (m * xg - Yrd) * Nv / mt)
    b4  = (dt * (Iz - Nrd) * Yr / mt) - (dt * (m * xg - Yrd) * Nr / mt)
    b5  = dt * (Iz - Nrd) * Yvv / mt
    b6  = -dt * (m * xg - Yrd) * Nrr / mt
    b7  = (dt * (m - Yvd) * Nv / mt) - (dt * (m * xg - Nvd) * Yv / mt)
    b8  = (dt * (m - Yvd) * Nr / mt) - (dt * (m * xg - Nvd) * Yr / mt)
    b9  = -dt * (m * xg - Nvd) * Yvv / mt
    b10 = dt * (m - Yvd) * Nrr / mt

    tau = np.array([5, 10, 0]).reshape(3,1)

    steps = int(ts/dt)
    u0 = 0.0
    v0 = 0.0
    r0 = 0.0

    u, v, r = float(u0), float(v0), float(r0)

    uvr = np.array([[u, v, r]])

    trajectory = []

    for _ in range(steps):

        c = np.array([[a1*v*r + a2*r*r + b1*u + b2*abs(u)*u],
                      [a3*u*r + a4*u*v + b3*v + b4*r + b5*abs(v)*v + b6*abs(r)*r],
                      [a5*u*r + a6*u*v + b7*v + b8*r + b9*abs(v)*v + b10*abs(r)*r]])
        
        uvr = uvr + c.T + (B @ tau).T

        u, v, r = uvr.flatten()

        trajectory.append(uvr.flatten())

    return torch.tensor(trajectory)